In [ ]:
# variables with suffix '_n' denote that nudged data are being used
# variables with prefix 'd' denote derivative

# dependencies
using MovingWeightedLeastSquares
using Plots
using Cubature
using Interpolations

# initialization of pyplot plotting backend
pyplot();

In [ ]:
# initialize the dataset in this Jupyter notebook cell

# from, to, step
from = -4;
to = 4;
step = 0.2;
# a random number in interval (-ng/2, ng/2) will be added to each x in xs
ng = .2;
# function to be approximated
f = x -> sin(2x)/2;
# its derivative
df = x -> cos(2x);
# weight function
w = (d, e) -> exp(-d^2);
# degree of the polynomial used for approximation
deg = 3;
# eps of the weight function
eps = 3step;
# attempts for time measurement
time_tries = 10;

In [ ]:
rg = from:step:to
xs = collect(rg) # input sample data
# comment the line below, if repeated evaluation with the same nudges is needed
rnd_n = ng * rand(size(xs)) - ng/2;
xs_n = xs .+ rnd_n;
fs = [f(x) for x in xs]; # output sample data
fs_n = [f(x) for x in xs_n];
dfs = [df(x) for x in xs];
dfs_n = [df(x) for x in xs_n];

# creation of approximation objects
# nv = mwls_naive(xs, fs, eps, w, maxDegree = deg);
# nv_n = mwls_naive(xs_n, fs_n, eps, w, maxDegree = deg);
kd = mwls_kd(xs, fs, eps, w, maxDegree = deg);
kd_n = mwls_kd(xs_n, fs_n, eps, w, maxDegree = deg);
cll = mwls_cll(xs, fs, eps, w, maxDegree = deg);
cll_n = mwls_cll(xs_n, fs_n, eps, w, maxDegree = deg);

In [ ]:
function plot_func(xs::Vector{T}, ys::Vector{T}, savename::String; ylabel::String = "", step::Real = 0.01) where {T <: Real}
    p = plot(xs, ys, ylabel = ylabel, legend = false)
    savefig(p, "$savename.pdf")
    p
end

In [ ]:
function plot_func(b::Real, e::Real, f, savename::String; ylabel::String = "", step::Real = 0.01)
    xs = collect(b:step:e)
    fs = [f(x) for x in xs]
    plot_func(xs, fs, savename, ylabel = ylabel)
end

In [ ]:
function plot_func(b::Real, e::Real, f::MwlsObject, reff, savename::String; ylabel::String = "", step::Real = 0.01)
    xs = collect(b:step:e)
    fs = [norm(f(x) - reff(x)) for x in xs]
    plot_func(xs, fs, savename, ylabel = ylabel)
end

In [ ]:
function plot_func(b::Real, e::Real, f, reff, savename::String; ylabel::String = "", step::Real = 0.01)
    xs = collect(b:step:e)
    fs = [norm(f[x] - reff(x)) for x in xs]
    plot_func(xs, fs, savename, ylabel = ylabel)
end

In [ ]:
function plot_dfunc_err(b::Real, e::Real, f::MwlsObject, reff, savename::String; ylabel::String = "", step::Real = 0.01)
    xs = collect(b:step:e)
    fs = [norm(mwls_diff(f, x, 1) - reff(x)) for x in xs]
    plot_func(xs, fs, savename, ylabel = ylabel)
end

In [ ]:
function plot_dfunc_err(b::Real, e::Real, f, reff, savename::String; ylabel::String = "", step::Real = 0.01)
    xs = collect(b:step:e)
    fs = [norm(gradient(f, x) - reff(x)) for x in xs]
    plot_func(xs, fs, savename, ylabel = ylabel)
end

In [ ]:
function errfunc(obj::MwlsObject, reffunc::Function, b, e)
    hquadrature(x -> abs(obj(x) - reffunc(x))/(e - b), b, e, abstol = 1e-8)
end

In [ ]:
function derrfunc(obj::MwlsObject, reffunc::Function, b, e)
    hquadrature(x -> abs(mwls_diff(obj, x, 1) - reffunc(x))/(e - b), b, e, abstol = 1e-8 )
end

In [ ]:
# graph of the original function
plot_func(from, to, f, "orig")

In [ ]:
plot_func(from, to, df, "der")

In [ ]:
plot_func(from, to, kd, f, "kd-err")

In [ ]:
# error of the approximation
errfunc(kd, f, from, to)

In [ ]:
plot_func(from, to, cll_n, f, "cll-err-n")

In [ ]:
errfunc(cll_n, f, from, to)

In [ ]:
plot_dfunc_err(from, to, cll, df, "dcll-err")

In [ ]:
derrfunc(cll, df, from, to)

In [ ]:
plot_dfunc_err(from, to, kd_n, df, "dkd-err-n")

In [ ]:
derrfunc(kd_n, df, from, to)

In [ ]:
function interpolations_errfunc(obj, reffunc, b, e; tol::Real = 1e-8)
    hquadrature(x -> abs(obj[x] - reffunc(x))/(e - b), b, e, abstol = tol)
end

In [ ]:
function interpolations_derrfunc(obj, reffunc, b, e; tol::Real = 1e-8)
    hquadrature(x -> abs(gradient(obj, x)[1] - reffunc(x))/(e - b), b, e, abstol = tol)
end

In [ ]:
# init of the interpolation object from Interpolations.jl
itp = scale(interpolate(fs, BSpline(Quadratic(Free())), OnGrid()), rg);

In [ ]:
# init of the interpolation object using nudged outputs
itp_n = scale(interpolate(fs_n, BSpline(Quadratic(Free())), OnGrid()), rg);

In [ ]:
plot_func(from, to, itp, f, "bsplines-err")

In [ ]:
interpolations_errfunc(itp, f, from, to; tol = 1e-20)

In [ ]:
plot_dfunc_err(from, to, itp, df, "dbsplines-err")

In [ ]:
interpolations_derrfunc(itp, df, from, to)

In [ ]:
plot_func(from, to, itp_n, f, "bsplines-err-n")

In [ ]:
interpolations_errfunc(itp_n, f, from, to)

In [ ]:
plot_dfunc_err(from, to, itp_n, df, "dbsplines-err-n")

In [ ]:
interpolations_derrfunc(itp_n, df, from, to)